In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! cp -r drive/MyDrive/tensorflow_constrained_optimization/tensorflow_constrained_optimization/tensorflow_constrained_optimization .

In [ ]:
! cp drive/MyDrive/new_Recidivism.csv .

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import warnings
from six.moves import xrange
from platform import python_version

In [ ]:
import tensorflow.compat.v1 as tf
import tensorflow_constrained_optimization as tfco
import matplotlib.pyplot as plt

tf.disable_eager_execution()

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('new_Recidivism.csv')

In [ ]:
s_data = df.Race.values

In [ ]:
s_data.shape

(12578,)

In [ ]:
s_data

array([-1, -1, -1, ...,  1,  1,  1])

In [ ]:
y_data = df.Recidivism.values

In [ ]:
#x_data_df = df.drop(['Race'], axis=1)

In [ ]:
x_df_labelled = df.rename(columns={'Recidivism': 'label'})

In [ ]:
x_df_labelled = x_df_labelled[list(x_df_labelled.keys())[1:]]

In [ ]:
x_df_labelled

,Release Type_Discharged - Expiration of Sentence,Release Type_Discharged – End of Sentence,Release Type_Parole,Release Type_Parole Granted,Release Type_Paroled to Detainer - INS,Release Type_Paroled to Detainer - Iowa,Release Type_Paroled to Detainer - Out of State,Release Type_Paroled to Detainer - U.S. Marshall,Release Type_Paroled w/Immediate Discharge,Release Type_Released to Special Sentence,Release Type_Special Sentence,Offense Classification_A Felony,Offense Classification_Aggravated Misdemeanor,Offense Classification_B Felony,Offense Classification_C Felony,Offense Classification_D Felony,Offense Classification_Felony - Enhanced,Offense Classification_Felony - Enhancement to Original Penalty,Offense Classification_Felony - Mandatory Minimum,Offense Classification_Other Felony,Offense Classification_Other Felony (Old Code),Offense Classification_Serious Misdemeanor,Offense Classification_Sexual Predator Community Supervision,Offense Classification_Simple Misdemeanor,Offense Classification_Special Sentence 2005,Offense Type_Drug,Offense Type_Other,Offense Type_Property,Offense Type_Public Order,Offense Type_Violent,Offense Subtype_Alcohol,Offense Subtype_Animals,Offense Subtype_Arson,Offense Subtype_Assault,Offense Subtype_Burglary,Offense Subtype_Drug Possession,Offense Subtype_Flight/Escape,Offense Subtype_Forgery/Fraud,Offense Subtype_Kidnap,Offense Subtype_Murder/Manslaughter,Offense Subtype_OWI,Offense Subtype_Other Criminal,Offense Subtype_Other Drug,Offense Subtype_Other Public Order,Offense Subtype_Other Violent,Offense Subtype_Prostitution/Pimping,Offense Subtype_Robbery,Offense Subtype_Sex,Offense Subtype_Sex Offender Registry/Residency,Offense Subtype_Special Sentence Revocation,Offense Subtype_Stolen Property,Offense Subtype_Theft,Offense Subtype_Traffic,Offense Subtype_Trafficking,Offense Subtype_Vandalism,Offense Subtype_Weapons,Race,label
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,-1,1
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,1
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12573,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1
12574,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1
12575,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1
12576,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1


In [ ]:
PROTECTED_COLUMNS = ['Race']
FEATURE_NAMES = list(x_df_labelled.keys())[:-1]
LABEL_COLUMN = 'label'

In [ ]:
PROTECTED_COLUMNS

['Race']

In [ ]:
FEATURE_NAMES

['Release Type_Discharged - Expiration of Sentence',
 'Release Type_Discharged – End of Sentence',
 'Release Type_Parole',
 'Release Type_Parole Granted',
 'Release Type_Paroled to Detainer - INS',
 'Release Type_Paroled to Detainer - Iowa',
 'Release Type_Paroled to Detainer - Out of State',
 'Release Type_Paroled to Detainer - U.S. Marshall',
 'Release Type_Paroled w/Immediate Discharge',
 'Release Type_Released to Special Sentence',
 'Release Type_Special Sentence',
 'Offense Classification_A Felony',
 'Offense Classification_Aggravated Misdemeanor',
 'Offense Classification_B Felony',
 'Offense Classification_C Felony',
 'Offense Classification_D Felony',
 'Offense Classification_Felony - Enhanced',
 'Offense Classification_Felony - Enhancement to Original Penalty',
 'Offense Classification_Felony - Mandatory Minimum',
 'Offense Classification_Other Felony',
 'Offense Classification_Other Felony (Old Code)',
 'Offense Classification_Serious Misdemeanor',
 'Offense Classification_Se

In [ ]:
class Model(object):
    def __init__(self,
                 tpr_max_diff=0):
        tf.random.set_random_seed(123)
        self.tpr_max_diff = tpr_max_diff
        num_features = len(FEATURE_NAMES)
        self.features_placeholder = tf.placeholder(
            tf.float32, shape=(None, num_features), name='features_placeholder')
        self.labels_placeholder = tf.placeholder(
            tf.float32, shape=(None, 1), name='labels_placeholder')
        self.protected_placeholders = [tf.placeholder(tf.float32, shape=(None, 1), name=attribute+"_placeholder") for attribute in PROTECTED_COLUMNS]
        # We use a linear model.
        self.predictions_tensor = tf.layers.dense(inputs=self.features_placeholder, units=1, activation=None)


    def build_train_op(self,
                       learning_rate,
                       unconstrained=False):
        #print(PROTECTED_COLUMNS)
        ctx = tfco.rate_context(self.predictions_tensor, self.labels_placeholder)
        positive_slice = ctx.subset(self.labels_placeholder > 0) 
        overall_tpr = tfco.positive_prediction_rate(positive_slice)
        print("overall")
        print(overall_tpr)
        constraints = []
        if not unconstrained:
            for placeholder in self.protected_placeholders:
                slice_tpr = tfco.positive_prediction_rate(ctx.subset((placeholder > 0) & (self.labels_placeholder > 0)))
                constraints.append(slice_tpr >= overall_tpr - self.tpr_max_diff)
        #print("constraints")
        #print(constraints)
        mp = tfco.RateMinimizationProblem(tfco.error_rate(ctx), constraints)
        opt = tfco.ProxyLagrangianOptimizerV1(tf.train.AdamOptimizer(learning_rate))
        self.train_op = opt.minimize(mp)
        return self.train_op
  
    def feed_dict_helper(self, dataframe):
        feed_dict = {self.features_placeholder:
                  dataframe[FEATURE_NAMES],
              self.labels_placeholder:
                  dataframe[[LABEL_COLUMN]],}
        #print("feed dict", feed_dict)
        for i, protected_attribute in enumerate(PROTECTED_COLUMNS):
            #print("i, protected_attribute")
            #print(i, protected_attribute)
            #print("dataframe")
            #print(dataframe)
            feed_dict[self.protected_placeholders[i]] = dataframe[[protected_attribute]]
            #print("hello")
        return feed_dict

In [ ]:
def training_generator(model,
                       train_df,
                       test_df,
                       minibatch_size,
                       num_iterations_per_loop=1,
                       num_loops=1):
    random.seed(31337)
    num_rows = train_df.shape[0]
    minibatch_size = min(minibatch_size, num_rows)
    permutation = list(range(train_df.shape[0]))
    random.shuffle(permutation)

    session = tf.Session()
    session.run((tf.global_variables_initializer(),
               tf.local_variables_initializer()))

    minibatch_start_index = 0
    for n in xrange(num_loops):
        for _ in xrange(num_iterations_per_loop):
            minibatch_indices = []
            while len(minibatch_indices) < minibatch_size:
                minibatch_end_index = (
                minibatch_start_index + minibatch_size - len(minibatch_indices))
                if minibatch_end_index >= num_rows:
                    minibatch_indices += range(minibatch_start_index, num_rows)
                    minibatch_start_index = 0
                else:
                    minibatch_indices += range(minibatch_start_index, minibatch_end_index)
                    minibatch_start_index = minibatch_end_index
                    
            session.run(
                  model.train_op,
                  feed_dict=model.feed_dict_helper(
                      train_df.iloc[[permutation[ii] for ii in minibatch_indices]]))

        train_predictions = session.run(
            model.predictions_tensor,
            feed_dict=model.feed_dict_helper(train_df))
        test_predictions = session.run(
            model.predictions_tensor,
            feed_dict=model.feed_dict_helper(test_df))

        yield (train_predictions, test_predictions)

In [ ]:
def error_rate(predictions, labels):
    signed_labels = (
      (labels > 0).astype(np.float32) - (labels <= 0).astype(np.float32))
    numerator = (np.multiply(signed_labels, predictions) <= 0).sum()
    denominator = predictions.shape[0]
    return float(numerator) / float(denominator)


def positive_prediction_rate(predictions, subset):
    numerator = np.multiply((predictions > 0).astype(np.float32),
                          (subset > 0).astype(np.float32)).sum()
    denominator = (subset > 0).sum()
    return float(numerator) / float(denominator)

def tpr(df):
    """Measure the true positive rate."""
    fp = sum((df['predictions'] >= 0.0) & (df[LABEL_COLUMN] > 0.5))
    ln = sum(df[LABEL_COLUMN] > 0.5)
    return float(fp) / float(ln)

def _get_error_rate_and_constraints(df, tpr_max_diff):
    """Computes the error and fairness violations."""
    error_rate_local = error_rate(df[['predictions']], df[[LABEL_COLUMN]])
    overall_tpr = tpr(df)
    return error_rate_local, [(overall_tpr - tpr_max_diff) - tpr(df[df[protected_attribute] > 0.5]) for protected_attribute in PROTECTED_COLUMNS]

def _get_exp_error_rate_constraints(cand_dist, error_rates_vector, constraints_matrix):
    """Computes the expected error and fairness violations on a randomized solution."""
    expected_error_rate = np.dot(cand_dist, error_rates_vector)
    expected_constraints = np.matmul(cand_dist, constraints_matrix)
    return expected_error_rate, expected_constraints

def training_helper(model,
                    train_df,
                    test_df,
                    minibatch_size,
                    num_iterations_per_loop=1,
                    num_loops=1):
    train_error_rate_vector = []
    train_constraints_matrix = []
    test_error_rate_vector = []
    test_constraints_matrix = []
    for train, test in training_generator(
      model, train_df, test_df, minibatch_size, num_iterations_per_loop,
      num_loops):

        train_df['predictions'] = train
        test_df['predictions'] = test

        train_error_rate, train_constraints = _get_error_rate_and_constraints(
          train_df, model.tpr_max_diff)
        train_error_rate_vector.append(train_error_rate)
        train_constraints_matrix.append(train_constraints)

        test_error_rate, test_constraints = _get_error_rate_and_constraints(
            test_df, model.tpr_max_diff)
        test_error_rate_vector.append(test_error_rate)
        test_constraints_matrix.append(test_constraints)

    return (train_error_rate_vector, train_constraints_matrix, test_error_rate_vector, test_constraints_matrix)

In [ ]:
train_size = 8000

In [ ]:
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_df_labelled, y_data, s_data, train_size=train_size, shuffle=True)

In [ ]:
model = Model(tpr_max_diff=0.05)
results = model.build_train_op(0.01, unconstrained=True)

# training_helper returns the list of errors and violations over each epoch.
train_errors, train_violations, test_errors, test_violations = training_helper(
      model,
      x_train,
      x_test,
      100,
      num_iterations_per_loop=326,
      num_loops=40)

In [ ]:
print("Train Error", train_errors[-1])
print("Train Violation", max(train_violations[-1]))

print("Test Error", test_errors[-1])
print("Test Violation", max(test_violations[-1]))

Train Error 0.286875
Train Violation 0.12802400056722166
Test Error 0.2885539536915684
Test Violation 0.13530641425359502


In [ ]:
! cp drive/MyDrive/SearchFair-master/examples/utils.py .

In [ ]:
import utils as ut

In [ ]:
s_test

array([ 1, -1, -1, ..., -1, -1, -1])

In [ ]:
def print_clf_stats(pred_x_train, pred_x_test, x_train, x_test, y_train, y_test, s_train, s_test):
    train_acc = ut.get_accuracy(np.sign(pred_x_train), y_train)
    print("train accuracy")
    print(train_acc)
    test_acc = ut.get_accuracy(np.sign(pred_x_test), y_test)
    print("test accuracy")
    print(test_acc)
    print(s_train)
    test_DDP, test_DEO = ut.compute_fairness_measures(np.sign(pred_x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(np.sign(pred_x_train), y_train, s_train)

    print(10*'-'+"Train"+10*'-')
    print("Accuracy: %0.4f%%" % (train_acc * 100))
    print("DDP: %0.4f%%" % (train_DDP * 100), "DEO: %0.4f%%" % (train_DEO * 100))
    print(10*'-'+"Test"+10*'-')
    print("Accuracy: %0.4f%%" % (test_acc * 100))
    print("DDP: %0.4f%%" % (test_DDP * 100), "DEO: %0.4f%%" % (test_DEO * 100))

In [ ]:
print_clf_stats(x_train['predictions'], x_test['predictions'], x_train, x_test, y_train, y_test, s_train, s_test)

train accuracy
0.713125
test accuracy
0.7114460463084317
----------Train----------
Accuracy: 71.3125%
DDP: -17.2836% DEO: -33.3408%
----------Test----------
Accuracy: 71.1446%
DDP: -18.5567% DEO: -34.9631%


In [ ]:
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_df_labelled, y_data, s_data, train_size=train_size, shuffle=True)

In [ ]:
model = Model(tpr_max_diff=0.05)
model.build_train_op(0.01, unconstrained=False)

# training_helper returns the list of errors and violations over each epoch.
train_errors, train_violations, test_errors, test_violations = training_helper(
      model,
      x_train,
      x_test,
      100,
      num_iterations_per_loop=326,
      num_loops=40)

overall


In [ ]:
print_clf_stats(x_train['predictions'], x_test['predictions'], x_train, x_test, y_train, y_test, s_train, s_test)

train accuracy
0.672375
test accuracy
0.6703800786369594
[-1  1 -1 ...  1 -1  1]
----------Train----------
Accuracy: 67.2375%
DDP: -1.3442% DEO: -1.8100%
----------Test----------
Accuracy: 67.0380%
DDP: -0.5407% DEO: -1.9544%


In [ ]:
print("Train Error", train_errors[-1])
print("Train Violation", max(train_violations[-1]))

print("Test Error", test_errors[-1])
print("Test Violation", max(test_violations[-1]))

Train Error 0.327625
Train Violation -0.04041086543027002
Test Error 0.32961992136304064
Test Violation -0.0394998829084453


In [ ]:
x_train

,Release Type_Discharged - Expiration of Sentence,Release Type_Discharged – End of Sentence,Release Type_Parole,Release Type_Parole Granted,Release Type_Paroled to Detainer - INS,Release Type_Paroled to Detainer - Iowa,Release Type_Paroled to Detainer - Out of State,Release Type_Paroled to Detainer - U.S. Marshall,Release Type_Paroled w/Immediate Discharge,Release Type_Released to Special Sentence,Release Type_Special Sentence,Offense Classification_A Felony,Offense Classification_Aggravated Misdemeanor,Offense Classification_B Felony,Offense Classification_C Felony,Offense Classification_D Felony,Offense Classification_Felony - Enhanced,Offense Classification_Felony - Enhancement to Original Penalty,Offense Classification_Felony - Mandatory Minimum,Offense Classification_Other Felony,Offense Classification_Other Felony (Old Code),Offense Classification_Serious Misdemeanor,Offense Classification_Sexual Predator Community Supervision,Offense Classification_Simple Misdemeanor,Offense Classification_Special Sentence 2005,Offense Type_Drug,Offense Type_Other,Offense Type_Property,Offense Type_Public Order,Offense Type_Violent,Offense Subtype_Alcohol,Offense Subtype_Animals,Offense Subtype_Arson,Offense Subtype_Assault,Offense Subtype_Burglary,Offense Subtype_Drug Possession,Offense Subtype_Flight/Escape,Offense Subtype_Forgery/Fraud,Offense Subtype_Kidnap,Offense Subtype_Murder/Manslaughter,Offense Subtype_OWI,Offense Subtype_Other Criminal,Offense Subtype_Other Drug,Offense Subtype_Other Public Order,Offense Subtype_Other Violent,Offense Subtype_Prostitution/Pimping,Offense Subtype_Robbery,Offense Subtype_Sex,Offense Subtype_Sex Offender Registry/Residency,Offense Subtype_Special Sentence Revocation,Offense Subtype_Stolen Property,Offense Subtype_Theft,Offense Subtype_Traffic,Offense Subtype_Trafficking,Offense Subtype_Vandalism,Offense Subtype_Weapons,Race,label,predictions
1867,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,-0.050030
9770,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1.261763
3730,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-0.541233
4782,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,-1.026367
10338,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,-1.053607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6599,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1.170241
10918,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,-1.261763
9374,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,-1,-0.957010
4040,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-0.793083


In [ ]:
x_train_cp = x_train.copy()

In [ ]:
rslt_df = x_train_cp[x_train_cp['Race'] == 1]

In [ ]:
rslt_df.shape

(5066, 59)

In [ ]:
rslt_2 = rslt_df[rslt_df['predictions'] > 0]

In [ ]:
rslt_2.shape

(165, 59)

In [ ]:
x_train_cp = x_train.copy()
rslt_df = x_train_cp[x_train_cp['Race'] == -1]


In [ ]:
rslt_df.shape

(2934, 59)

In [ ]:
rslt_2 = rslt_df[rslt_df['predictions'] > 0]

In [ ]:
rslt_2.shape

(135, 59)

In [ ]:
135/2934

0.046012269938650305

In [ ]:
165/5066

0.03257007500986972

In [ ]:
baselineTrainDEO = []
baselineTrainDDP = []
baselineTestDEO = []
baselineTestDDP = []
baselineTrainAcc = []
baselineTestAcc = []

constraintTrainDEO = []
constraintTrainDDP = []
constraintTestDEO = []
constraintTestDDP = []
constraintTrainAcc = []
constraintTestAcc = []

In [ ]:
def print_clf_stats_baseline(pred_x_train, pred_x_test, x_train, x_test, y_train, y_test, s_train, s_test):
    train_acc = ut.get_accuracy(np.sign(pred_x_train), y_train)
    #print("train accuracy")
    #print(train_acc)
    test_acc = ut.get_accuracy(np.sign(pred_x_test), y_test)
    #print("test accuracy")
    #print(test_acc)
    test_DDP, test_DEO = ut.compute_fairness_measures(np.sign(pred_x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(np.sign(pred_x_train), y_train, s_train)

    #print(10*'-'+"Train"+10*'-')
    #print("Accuracy: %0.4f%%" % (train_acc * 100))
    #print("DDP: %0.4f%%" % (train_DDP * 100), "DEO: %0.4f%%" % (train_DEO * 100))
    #print(10*'-'+"Test"+10*'-')
    #print("Accuracy: %0.4f%%" % (test_acc * 100))
    #print("DDP: %0.4f%%" % (test_DDP * 100), "DEO: %0.4f%%" % (test_DEO * 100))
    baselineTrainDEO.append(train_DEO)
    baselineTrainDDP.append(train_DDP)
    baselineTestDEO.append(test_DEO)
    baselineTestDDP.append(test_DDP)
    baselineTrainAcc.append(train_acc)
    baselineTestAcc.append(test_acc)

In [ ]:
def print_clf_stats_constraint(pred_x_train, pred_x_test, x_train, x_test, y_train, y_test, s_train, s_test):
    train_acc = ut.get_accuracy(np.sign(pred_x_train), y_train)
    #print("train accuracy")
    #print(train_acc)
    test_acc = ut.get_accuracy(np.sign(pred_x_test), y_test)
    print("test accuracy")
    #print(test_acc)
    test_DDP, test_DEO = ut.compute_fairness_measures(np.sign(pred_x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(np.sign(pred_x_train), y_train, s_train)

    #print(10*'-'+"Train"+10*'-')
    #print("Accuracy: %0.4f%%" % (train_acc * 100))
    #print("DDP: %0.4f%%" % (train_DDP * 100), "DEO: %0.4f%%" % (train_DEO * 100))
    #print(10*'-'+"Test"+10*'-')
    #print("Accuracy: %0.4f%%" % (test_acc * 100))
    #print("DDP: %0.4f%%" % (test_DDP * 100), "DEO: %0.4f%%" % (test_DEO * 100))
    constraintTrainDEO.append(train_DEO)
    constraintTrainDDP.append(train_DDP)
    constraintTestDEO.append(test_DEO)
    constraintTestDDP.append(test_DDP)
    constraintTrainAcc.append(train_acc)
    constraintTestAcc.append(test_acc)



In [ ]:
for i in range(5):
  x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_df_labelled, y_data, s_data, train_size=train_size, shuffle=True)
  model = Model(tpr_max_diff=0.05)
  results = model.build_train_op(0.01, unconstrained=True)

  # training_helper returns the list of errors and violations over each epoch.
  train_errors, train_violations, test_errors, test_violations = training_helper(
        model,
        x_train,
        x_test,
        100,
        num_iterations_per_loop=326,
        num_loops=40)
  
  print_clf_stats_baseline(x_train['predictions'], x_test['predictions'], x_train, x_test, y_train, y_test, s_train, s_test)

  print("iter "+str(i + 1)+ 'finished')

iter 1finished
iter 2finished
iter 3finished
iter 4finished
iter 5finished


In [ ]:
for i in range(5):
  x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_df_labelled, y_data, s_data, train_size=train_size, shuffle=True)
  model = Model(tpr_max_diff=0.05)
  model.build_train_op(0.01, unconstrained=False)

  # training_helper returns the list of errors and violations over each epoch.
  train_errors, train_violations, test_errors, test_violations = training_helper(
        model,
        x_train,
        x_test,
        100,
        num_iterations_per_loop=326,
        num_loops=40)
  
  print_clf_stats_constraint(x_train['predictions'], x_test['predictions'], x_train, x_test, y_train, y_test, s_train, s_test)
  print("iter "+str(i + 1)+ 'finished')

test accuracy
iter 1finished
test accuracy
iter 2finished
test accuracy
iter 3finished
test accuracy
iter 4finished
test accuracy
iter 5finished


In [ ]:
from statistics import mean

In [ ]:
baselineTrainDEO_mean = mean(baselineTrainDEO)
baselineTrainDDP_mean = mean(baselineTrainDDP)
baselineTestDEO_mean = mean(baselineTestDEO)
baselineTestDDP_mean = mean(baselineTestDDP)
baselineTrainAcc_mean = mean(baselineTrainAcc)
baselineTestAcc_mean = mean(baselineTestAcc)

constraintTrainDEO_mean = mean(constraintTrainDEO)
constraintTrainDDP_mean = mean(constraintTrainDDP)
constraintTestDEO_mean = mean(constraintTestDEO)
constraintTestDDP_mean = mean(constraintTestDDP)
constraintTrainAcc_mean = mean(constraintTrainAcc)
constraintTestAcc_mean = mean(constraintTestAcc)

In [ ]:
print("baseline averages:")
print(10*'-'+"Train"+10*'-')
print("Accuracy: %0.4f%%" % (baselineTrainAcc_mean * 100))
print("DDP: %0.4f%%" % (baselineTrainDDP_mean * 100), "DEO: %0.4f%%" % (baselineTrainDEO_mean * 100))
print(10*'-'+"Test"+10*'-')
print("Accuracy: %0.4f%%" % (baselineTestAcc_mean * 100))
print("DDP: %0.4f%%" % (baselineTestDDP_mean * 100), "DEO: %0.4f%%" % (baselineTestDEO_mean * 100))

print("constraint averages:")
print(10*'-'+"Train"+10*'-')
print("Accuracy: %0.4f%%" % (constraintTrainAcc_mean * 100))
print("DDP: %0.4f%%" % (constraintTrainDDP_mean * 100), "DEO: %0.4f%%" % (constraintTrainDEO_mean * 100))
print(10*'-'+"Test"+10*'-')
print("Accuracy: %0.4f%%" % (constraintTestAcc_mean * 100))
print("DDP: %0.4f%%" % (constraintTestDDP_mean * 100), "DEO: %0.4f%%" % (constraintTestDEO_mean * 100))

baseline averages:
----------Train----------
Accuracy: 66.1400%
DDP: -39.1654% DEO: -52.2175%
----------Test----------
Accuracy: 65.6335%
DDP: -39.4769% DEO: -52.5338%
constraint averages:
----------Train----------
Accuracy: 63.5275%
DDP: -2.8828% DEO: -6.0429%
----------Test----------
Accuracy: 62.9642%
DDP: -2.3914% DEO: -5.8203%


In [ ]:
print(constraintTestDEO, constraintTestDDP)

[-0.10142245707099262, 0.006471568971568972, -0.3047112067541612, 0.0013622818220519368, 0.0029653702614347572, -0.032629245921743474, -0.042695360291118985, -0.057307431374595555, 0.03193544319757912, -0.08600328781829772] [-0.024521809510261616, 0.0029246705728200946, -0.1187667482502151, 0.00016188431671432572, 0.0004249764745165893, -0.0175382124108706, -0.022681093867453025, -0.03044090515019254, 0.015916422023537154, -0.044618269792816787]


In [ ]:
print_clf_stats(x_train['predictions'], x_test['predictions'], x_train, x_test, y_train, y_test, s_train, s_test)

train accuracy
0.6775
test accuracy
0.6797728265618174
[1 1 1 ... 1 1 1]
----------Train----------
Accuracy: 67.7500%
DDP: -6.5634% DEO: -12.2500%
----------Test----------
Accuracy: 67.9773%
DDP: -4.4618% DEO: -8.6003%
